# 作業  

嘗試用 keras 的 DepthwiseConv2D 等 layers 實做 Inverted Residual Block:  
   - depthwise's filter shape 爲 (3,3), padding = same
   - 不需要給 alpha, depth multiplier 參數
   - expansion 因子爲 6

# 載入套件

In [1]:
import keras
from keras.models import Input, Model
from keras.layers import DepthwiseConv2D, Conv2D, BatchNormalization, ReLU, Add
print( 'Keras 的版本:', keras.__version__ )

Keras 的版本: 2.4.3


# 定義 Separable Convolution 函數

In [2]:
def InvertedRes(input, expansion):
    '''
    Args:
        input: input tensor
        expansion: expand filters size
    Output:
        output: output tensor
    '''
    # Expansion Layer
    x = Conv2D(expansion * 3,(1,1),padding="same")(input)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Depthwise Convolution
    x = DepthwiseConv2D((3,3), padding="same")(input)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    # Add Layer
    x = Add()([input,x])

    return x

# 建構模型

In [3]:
input = Input((64, 64, 3))
output = InvertedRes(input, 6)
model = Model(inputs=input, outputs=output)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
depthwise_conv2d (DepthwiseConv (None, 64, 64, 3)    30          input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 3)    12          depthwise_conv2d[0][0]           
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 64, 64, 3)    0           batch_normalization_1[0][0]      
_______________________________________________________________________________________